In [1]:
import librosa
import librosa.display as librosa_display
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import os
import time
from torchaudio.functional import compute_deltas
import torch

In [2]:
def prepare_RAVDESS_DS(path_audios):
    wav_paths, emotions, actors = [], [], []
    for path in tqdm(Path(path_audios).glob("*/*.wav")):
        name = str(path).split('/')[-1].split('.')[0]
        label = int(name.split("-")[2]) - 1  # Start emotions in 0
        actor = int(name.split("-")[-1])

        try:
            wav_paths.append(path)
            emotions.append(label)
            actors.append(actor)
        except Exception as e:
            # print(str(path), e)
            pass
        
    return wav_paths, emotions, actors

In [3]:
wav_paths, emotions, actors = prepare_RAVDESS_DS('RAVDESS')

1440it [00:00, 59996.60it/s]


In [4]:
sample_rate = 16000

frame_length = 0.05
frame_stride = 0.0125

In [5]:
def get_features(paths):
    data = []
    sz = []
    for i, path, in tqdm(enumerate(paths), desc='get features, size.....'):
        input_nfft = int(round(sample_rate*frame_length))
        input_stride = int(round(sample_rate*frame_stride))
        
        y, _ = librosa.load(path, sr=sample_rate)
        
        S = librosa.feature.melspectrogram(y=y, sr=sample_rate, n_mels=128, n_fft=input_nfft, window='hamming', hop_length=input_stride)
        log_S = librosa.power_to_db(S, ref=np.max)
        
        mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=39)
        
        data.append(mfcc)
        sz.append(mfcc.shape[1])
        
    return data, sz

In [6]:
features, sz = get_features(wav_paths)

get features, size.....: 1440it [01:37, 14.79it/s]


In [7]:
def make_df(features, emotions, actors, sz):
    data = []
    
    for i, feature in tqdm(enumerate(features), desc='Make dataframe.....'):
        mfcc_feature = np.zeros((39, sz))
        mfcc_feature = feature[:,:sz]
        
        data.append({
            'feature': mfcc_feature,
            'emotion': emotions[i],
            'actor': actors[i]
        })
    
    return pd.DataFrame(data)    

In [8]:
df = make_df(features, emotions, actors, min(sz))

Make dataframe.....: 1440it [00:00, 240026.93it/s]


In [9]:
def get_deltas(feature):
    f = np.expand_dims(feature, -1)
    f = torch.Tensor(f)
    
    delta = compute_deltas(f, win_length=7)
    delta2 = compute_deltas(delta, win_length=7)
    
    ret = torch.cat([f, delta, delta2], dim=-1)
    
    return ret

In [10]:
def get_fold(fold, df):
    """
    Divide the data in train and test in a subject-wise 5-CV way. The division is generated before running the training
    of each fold.
    :param fold:[int] Fold to create the train and test sets [ranging from 0 - 4]
    :param df:[DataFrame] Dataframe with the complete list of files generated
    :param save_path:[str] Path to save the train.csv and test.csv per fold
    """
    
    actors_per_fold = {
        0: [2,5,14,15,16],
        1: [3, 6, 7, 13, 18],
        2: [10, 11, 12, 19, 20],
        3: [8, 17, 21, 23, 24],
        4: [1, 4, 9, 22],
    }

    df = df.loc[df['actor'].isin(actors_per_fold[fold])]
    
    X = np.array([data for data in df['feature']])
    y = np.array([data for data in df['emotion']])
    
    X = get_deltas(X)
    
    return np.array(X), np.array(y)

In [11]:
X_dataset = []
y_dataset = []
for fold in range(5):
    X, y = get_fold(fold, df)
    X_dataset.append(X)
    y_dataset.append(y)

In [12]:
X = X_dataset[0]
y = y_dataset[0]
for fold in range(1, 5):
    X = np.concatenate((X, X_dataset[fold]), axis=0)
    y = np.concatenate((y, y_dataset[fold]), axis=0)

os.makedirs('dataset', exist_ok=True)
with open('dataset/RAVDESS-deltas.npy', 'wb') as f:
    np.save(f, X)
    np.save(f, y)

In [13]:
print(X.shape)

(1440, 39, 235, 3)
